# Exploratory Data Analysis
## Import Packages

In [43]:
import pandas as pd
import numpy as np
import geopandas as gp

## Importa Data

Il primo DataFrame contiene i dati sui consumi delle linee elettriche di tutto il Trentino.  
Per ogni linea elettrica viene fornito il dato di consumo ogni 10 minuti per l'intero mese.  
I dati sono per il mese di Novembre 2013 (il processo verrà ripetuto per il mese di Dicembre 2013).


Il secondo DataFrame contiene i dati riguardanti le ubicazioni e linee elettriche presenti in ogni zona del Trentino.  
Per ogni zona del Trentino (identificata da *SQUAREID*) vengono associate le linee elettriche da lì passanti (*LINESET*) e per ogni combinazione di zona e linea elettrica viene associato il numero di ubicazioni presenti (*NR_UBI*)



In [44]:
set_df = pd.read_csv("./data/processed/SET-nov-2013.csv")
lin_df = pd.read_csv("./data/processed/line.csv")
set_df

,LINESET,DATE,POWER
0,DG1000420,01/11/2013 00:00,37.439999
1,DG1000420,01/11/2013 00:10,37.439999
2,DG1000420,01/11/2013 00:20,36.000000
3,DG1000420,01/11/2013 00:30,35.279999
4,DG1000420,01/11/2013 00:40,35.279999
...,...,...,...
794513,DG1056626,30/11/2013 23:10,46.549999
794514,DG1056626,30/11/2013 23:20,44.650002
794515,DG1056626,30/11/2013 23:30,44.599998
794516,DG1056626,30/11/2013 23:40,43.700001


Trasformiamo la colonna *DATE* in tre colonne separate che indicano il giorno (*DAY*), i minuti (*MIN*) e il giorno della settimana (*WEEK_DAY*).

In [45]:
set_df["DATE"] = pd.to_datetime(set_df["DATE"], format="%d/%m/%Y %H:%M", dayfirst=True)
set_df["DAY"] = pd.to_datetime(set_df["DATE"]).dt.day
set_df["MINUTE"] = pd.to_datetime(set_df["DATE"]).dt.hour * 60 + pd.to_datetime(set_df["DATE"]).dt.minute
set_df["WEEK_DAY"] = pd.to_datetime(set_df["DATE"]).dt.day_of_week
del set_df["DATE"]
set_df

,LINESET,POWER,DAY,MINUTE,DAY_ID
0,DG1000420,37.439999,1,0,4
1,DG1000420,37.439999,1,10,4
2,DG1000420,36.000000,1,20,4
3,DG1000420,35.279999,1,30,4
4,DG1000420,35.279999,1,40,4
...,...,...,...,...,...
794513,DG1056626,46.549999,30,1390,5
794514,DG1056626,44.650002,30,1400,5
794515,DG1056626,44.599998,30,1410,5
794516,DG1056626,43.700001,30,1420,5


Ora voglio associare i dati dei due DataFrame organizzarli in base alla zona

In [50]:
lin_df

,SQUAREID,LINESET,NR_UBI
0,155,DG1031221,4
1,272,DG1031221,72
2,273,DG1031221,21
3,276,DG1003117,34
4,277,DG1003117,3
...,...,...,...
2570,10983,DG1013504,5
2571,10984,DG1013504,2
2572,11097,DG1013503,1
2573,11098,DG1013503,6


In [60]:
temp_df1 = lin_df.groupby("LINESET")["NR_UBI"].sum().reset_index()
temp_df1 = temp_df1.rename(columns={"NR_UBI": "UBI_X_LINE"})
temp_df1

,LINESET,UBI_X_LINE
0,DG1000011,6
1,DG1000023,62
2,DG1000420,2133
3,DG1000421,1424
4,DG1000422,3423
...,...,...
195,DG1099206,105
196,DG1099207,95
197,DG1099303,39
198,DG1099502,3


In [61]:
temp_df2 = lin_df.groupby("SQUAREID")["NR_UBI"].sum().reset_index()
temp_df2 = temp_df2.rename(columns={"NR_UBI": "UBI_X_SQUARE"})
temp_df2

,SQUAREID,UBI_X_SQUARE
0,155,4
1,272,72
2,273,21
3,276,34
4,277,3
...,...,...
2015,10983,5
2016,10984,2
2017,11097,1
2018,11098,6
